my gpu

In [1]:
!nvidia-smi

Wed May  7 12:51:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.133.07             Driver Version: 566.07         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070 ...    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8             14W /  130W |      98MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")
sequence_to_classify = "Angela Merkel is a politician in Germany and leader of the CDU"
candidate_labels = ["politics", "economy", "entertainment", "environment"]
output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
print(output)

Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'sequence': 'Angela Merkel is a politician in Germany and leader of the CDU', 'labels': ['politics', 'economy', 'environment', 'entertainment'], 'scores': [0.9823215007781982, 0.007280226331204176, 0.005891900975257158, 0.004506299737840891]}


In [2]:
from datasets import load_dataset

ds = load_dataset("wikimedia/wikipedia", "20231101.en", streaming=True)

print(ds)

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

IterableDatasetDict({
    train: IterableDataset({
        features: ['id', 'url', 'title', 'text'],
        num_shards: 41
    })
})


In [3]:
# Access the first element of the IterableDataset
first_element = next(iter(ds["train"]))
print(first_element)

{'id': '12', 'url': 'https://en.wikipedia.org/wiki/Anarchism', 'title': 'Anarchism', 'text': 'Anarchism is a political philosophy and movement that is skeptical of all justifications for authority and seeks to abolish the institutions it claims maintain unnecessary coercion and hierarchy, typically including nation-states, and capitalism. Anarchism advocates for the replacement of the state with stateless societies and voluntary free associations. As a historically left-wing movement, this reading of anarchism is placed on the farthest left of the political spectrum, usually described as the libertarian wing of the socialist movement (libertarian socialism).\n\nHumans have lived in societies without formal hierarchies long before the establishment of states, realms, or empires. With the rise of organised hierarchical bodies, scepticism toward authority also rose. Although traces of anarchist ideas are found all throughout history, modern anarchism emerged from the Enlightenment. During

In [10]:
sentences = first_element["text"].split("\n")
print(sentences[0])

Anarchism is a political philosophy and movement that is skeptical of all justifications for authority and seeks to abolish the institutions it claims maintain unnecessary coercion and hierarchy, typically including nation-states, and capitalism. Anarchism advocates for the replacement of the state with stateless societies and voluntary free associations. As a historically left-wing movement, this reading of anarchism is placed on the farthest left of the political spectrum, usually described as the libertarian wing of the socialist movement (libertarian socialism).


In [13]:
text_to_classify = sentences[0]
labels = ["sport", "not sport"]
output = classifier(text_to_classify, labels, multi_label=False)
print(output['labels'])
print(output['scores'])

['not sport', 'sport']
[0.9894245266914368, 0.010575489141047001]


In [66]:
text_sport = '''
Amateur astronomy is a hobby where participants enjoy observing or imaging celestial objects in the sky using the unaided eye, binoculars, or telescopes. Even though scientific research may not be their primary goal, some amateur astronomers make contributions in doing citizen science, such as by monitoring variable stars, double stars, sunspots, or occultations of stars by the Moon or asteroids, or by discovering transient astronomical events, such as comets, galactic novae or supernovae in other galaxies.
Amateur astronomers do not use the field of astronomy as their primary source of income or support, and usually have no professional degree in astrophysics or advanced academic training in the subject.  Most amateurs are hobbyists, while others have a high degree of experience in astronomy and may often assist and work alongside professional astronomers. Many astronomers have studied the sky throughout history in an amateur framework; however, since the beginning of the twentieth century, professional astronomy has become an activity clearly distinguished from amateur astronomy and associated activities.
Amateur astronomers typically view the sky at night, when most celestial objects and astronomical events are visible, but others observe during the daytime by viewing the Sun and solar eclipses.  Some just look at the sky using nothing more than their eyes or binoculars, but more dedicated amateurs often use portable telescopes or telescopes situated in their private or club observatories. Amateurs can also join as members of amateur astronomical societies, which can advise, educate or guide them towards ways of finding and observing celestial objects. They can also promote the science of astronomy among the general public.
'''
labels = ["sport", "not sport"]
output = classifier(text_sport, labels, multi_label=False)
print(output['labels'])
print(output['scores'])

['not sport', 'sport']
[0.9222761988639832, 0.07772379368543625]


In [63]:
text_sport_fr = "La vie est belle."
labels = ["sport", "non sport"]
output = classifier(text_sport_fr, labels, multi_label=False)
print(output['labels'])
print(output['scores'])

['non sport', 'sport']
[0.8453111052513123, 0.15468890964984894]


In [57]:
text_sport_zh = "这是一棵树"
labels = ["运动", "非运动"]
output = classifier(text_sport_zh, labels, multi_label=False)
print(output['labels'])
print(output['scores'])

['非运动', '运动']
[0.9101300239562988, 0.08987002074718475]


In [39]:
empty_text = " "
labels = ["sport", "not sport"]
output = classifier(empty_text, labels, multi_label=False)
print(output['labels'])
print(output['scores'])

['not sport', 'sport']
[0.5762802362442017, 0.42371973395347595]


In [4]:
from transformers import pipeline

CLASSIFIER_MODEL = "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli"
MISSION = "zero-shot-classification"
classifier = pipeline(MISSION, model=CLASSIFIER_MODEL)

Device set to use cuda:0


In [15]:
def classify_batch(texts, classifier, labels):
    outputs = classifier(texts, labels, multi_label=False)
    results = []
    for output in outputs:
        is_positive = output['labels'][0] == labels[0] and output['scores'][0] > 0.75
        results.append(is_positive)
    return results

In [6]:
from datasets import load_dataset

ds_en = load_dataset("wikimedia/wikipedia", "20231101.en", streaming=True)
ds_fr = load_dataset("wikimedia/wikipedia", "20231101.fr", streaming=True)
ds_zh = load_dataset("wikimedia/wikipedia", "20231101.zh", streaming=True)

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

In [13]:
from itertools import islice
from tqdm import tqdm
def dataset_filter(ds, classifier, labels, max_length=100, batch_size=8):
    filtered_ds = []
    batch = []
    batch_meta = []

    for element in tqdm(islice(ds, max_length)):
        text = ''.join(element["text"].split("\n")[:5])
        if text:
            batch.append(text)
            batch_meta.append(element)

        if len(batch) == batch_size:
            results = classify_batch(batch, classifier, labels)
            filtered_ds.extend([meta for meta, keep in zip(batch_meta, results) if keep])
            batch, batch_meta = [], []

    # last batch
    if batch:
        results = classify_batch(batch, classifier, labels)
        filtered_ds.extend([meta for meta, keep in zip(batch_meta, results) if keep])

    return filtered_ds

In [17]:
ds_sport = []
labels_en = ["sport", "not sport"]
ds_sport = dataset_filter(ds_en["train"], classifier, labels_en, max_length=1000, batch_size=16)
print(len(ds_sport))

192it [00:16, 12.87it/s]

: 

In [10]:
print(ds_sport)

[{'id': '595', 'url': 'https://en.wikipedia.org/wiki/Andre%20Agassi', 'title': 'Andre Agassi', 'text': 'Andre Kirk Agassi ( ; born April 29, 1970) is an American former world No. 1 tennis player. He is an eight-time major champion and an Olympic gold medalist, as well as a runner-up in seven other majors. Agassi is widely considered one of the greatest tennis players of all time.\n\nAgassi is the second of five men to achieve the career Grand Slam in the Open Era and the fifth of eight overall to make the achievement. He is also the first of two men to achieve the career Golden Slam (career Grand Slam and Olympic gold medal), as well as the only man to win a career Super Slam (career Grand Slam, plus the Olympic gold medal and the year-end championships).\n\nAgassi was the first man to win all four singles majors on three different surfaces (hard, clay and grass), and remains the most recent American man to win the French Open (in 1999) and the Australian Open (in 2003). He also won 17